In [1]:
from __future__ import absolute_import, division, print_function

import numpy as np




from sklearn.cluster import DBSCAN
import gensim
import logging
import os
import codecs
import pandas as pd
import numpy as np
import json
from gensim.models import Doc2Vec
from gensim.models import word2vec
import nltk
from nltk import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer



D:\new\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
def devide_words(sentence):
    __tokenization_pattern = r'''(?x)          
            \$?\d+(?:\.\d+)?%?  
          | (?:[A-Z]\.)+        
          | \w+(?:-\w+)*        
          | \.\.\.              
          | [][.,;"'?():_`-]    
        '''
    tokenizer = nltk.tokenize.regexp.RegexpTokenizer(__tokenization_pattern)
    stopwords_set = set(stopwords.words('english'))
    stemmer = SnowballStemmer("english")

    raw_tokens = tokenizer.tokenize(sentence)
    alphabet_tokens = [token for token in raw_tokens if token.isalpha()]
    nostopwords_tokens = [token for token in alphabet_tokens if not token in stopwords_set]
    tokens = [str(stemmer.stem(token)) for token in nostopwords_tokens]
    return tokens

def generate_dataset(data):
    data_set = []
    id_sequence = []
    seq = 0
    for i in range(len(data)):
        if seq > 3000:
             break
        for j in range(len(data[i])):
            if seq > 3000:
                 break
            content = data[i][j][1][4]
            words = devide_words(content)
            data_set.append(words)
            id_sequence.append(str(seq))
            seq += 1

    return data_set,id_sequence

def generate_doc2vec_model(dataset_div):
    TaggededDocument = gensim.models.doc2vec.TaggedDocument
    x_train = []
    for i in range(len(dataset_div)):
        document = TaggededDocument(dataset_div[i],tags=[i])
        x_train.append(document)
    model = Doc2Vec(x_train,min_count=1,window=3, sample = 1e-3, nagative = 5, workers = 4)
    model.train(x_train,total_examples=model.corpus_count,epochs = 10)
    return model

def average(model,dataset_div):
    doc_vec = []
    error_place = []
    for i in range(len(dataset_div)):
        try:
            doc_vec.append(sum(model[dataset_div[i]])/len(dataset_div[i]))
        except:
            #print(i,"key error!")
            error_place.append(i)
    return doc_vec,error_place

def write_json_file(dataset_div,doc_vec,id_sequence,error_place):
    json_dict = {}
    print("error_place is:")
    print(error_place)
    j=0
    for i in range(len(dataset_div)):
        if i not in error_place:
            json_dict["record_"+id_sequence[i]] = doc_vec[j].tolist()
            j+=1
    file = open(outputpath+'/text_300.json', 'w', encoding='utf-8')
    json.dump(json_dict,file)

def read_json_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        s = json.loads(f.read())
    return s

In [3]:
def cluster_DBSCAN(doc_vec):
    clusterer = DBSCAN(eps=0.5, min_samples=1,metric_params=None,leaf_size=30, p=None, n_jobs=None)
    clusterer.fit(doc_vec)
    return clusterer

In [4]:
def write_label(doc_vec,pickle_data,error_place): #write cluster result to json

    filename_list = list(doc_vec)
    # print(filename_list)
    doc_vec = doc_vec.T
    clusterer = cluster_DBSCAN(doc_vec)

    print(clusterer.labels_)
    print(len(clusterer.labels_))

    for i in range(len(doc_vec)):
        # print(filename_list[i],'+',clusterer.labels_[i])
        pickle_data[i][0]=filename_list[i]
        # j = int(filename_list[i][7:])
        # print(j)
        if i < len(clusterer.labels_):
            if clusterer.labels_[i] == -1:
                text = '0'
            else:
                text = '1'
        pickle_data[i].append(text)
    for i in error_place:
        pickle_data[i][0] = "record_" + str(i)
        pickle_data[i].append(0)
    return clusterer,pickle_data

def rebuild_pickle_data(pickle_data):
    result = []
    seq = 0
    for i in range(len(pickle_data)):
        if seq > 3000:
             break
        for j in range(len(pickle_data[i])):
            if seq > 3000:
                 break
            result.append(["0",pickle_data[i][j][1][4]])
            seq += 1
    return result

import spacy



In [5]:
pickle_root = 'kosmos3.p'
'''
1. Import spacy (above), 
2. load the largest word vector dataset into variable nlp
3. create a docvec function which gets the individual word vectors of a document and averages them in numpy
this allows whatever is fed in (regardless of length) to return as a 300 dimensional vector'''

nlp = spacy.load('en_core_web_lg')

'''load the dataset'''
import pickle
outputpath = 'Output'
headlines = pickle.load(open(pickle_root, "rb" ))

In [6]:
print(len(headlines))
print(len(headlines[0]))
print(len(headlines[0][0]))
print(len(headlines[0][0][1]))
print(headlines[0][0])
print(headlines[0][0][1][4])

dataset_div, id_sequence = generate_dataset(headlines)
print(dataset_div[0])
# print(dataset_div[len(dataset_div)-1])
# print(headlines[4][len(dataset_div)-1-8000][1][4])
# print(len(dataset_div))
# print(id_sequence[len(dataset_div)-1])

model = generate_doc2vec_model(dataset_div)
doc_vec, error_place = average(model, dataset_div)
print(len(doc_vec))
# print(doc_vec[len(doc_vec)-1])
# print(type(doc_vec[len(doc_vec)-1]))

write_json_file(dataset_div, doc_vec, id_sequence, error_place)
doc_vec_json = read_json_file(outputpath+'/text_300.json')
print(doc_vec_json['record_0'])

doc_vec = pd.DataFrame(doc_vec_json)
print(doc_vec.head())
# print(doc_vec)
result = rebuild_pickle_data(headlines)
print(len(result))
clusterer, results = write_label(doc_vec, result, error_place)
test = pickle.dump(results, open(outputpath + "/text_300.p", "wb"))
results = pickle.load(open(outputpath + "/text_300.p", "rb"))
print(len(results))
print(results[0])

304
2000
2
5
(-1, [ObjectId('5c9ffec3c32dab0a9c7cff90'), 1553983515.0, 'https://www.bbc.co.uk/sport/golf/47760913', 'Danish debutant Bjerregaard stuns Woods at Match Play', '  Bjerregaard beat Henrik Stenson earlier on Saturday Debutant Lucas Bjerregaard carded five birdies and an eagle to end the hopes of Tiger Woods at the WGC Match Play. Woods had beaten Rory McIlroy earlier in the day but missed a four-foot par putt as Bjerregaard won one-up. In Sunday\'s semi-finals world number 52 Bjerregaard faces Matt Kuchar, who beat Sergio Garcia two up after controversy over a putt that was not conceded. Open champion Francesco Molinari surged to a 6&5 win over Kevin Na, who earlier ousted world number two Justin Rose. The Italian will play Kevin Kisner, who won three holes in a row to beat Louis Oosthuizen 2&1. Denmark\'s Bjerregaard, winner of the Dunhill Links last year for his second European Tour title, birdied the first hole to take the lead against Woods, ranked 14th. Inspired to try 

['bjerregaard', 'beat', 'henrik', 'stenson', 'earlier', 'saturday', 'debut', 'luca', 'bjerregaard', 'card', 'five', 'birdi', 'eagl', 'end', 'hope', 'tiger', 'wood', 'wgc', 'match', 'play', 'wood', 'beaten', 'rori', 'mcilroy', 'earlier', 'day', 'miss', 'par', 'putt', 'bjerregaard', 'in', 'sunday', 'world', 'number', 'bjerregaard', 'face', 'matt', 'kuchar', 'beat', 'sergio', 'garcia', 'two', 'controversi', 'putt', 'conced', 'open', 'champion', 'francesco', 'molinari', 'surg', 'win', 'kevin', 'na', 'earlier', 'oust', 'world', 'number', 'two', 'justin', 'rose', 'the', 'italian', 'play', 'kevin', 'kisner', 'three', 'hole', 'row', 'beat', 'loui', 'oosthuizen', 'denmark', 'bjerregaard', 'winner', 'dunhil', 'link', 'last', 'year', 'second', 'european', 'tour', 'titl', 'birdi', 'first', 'hole', 'take', 'lead', 'wood', 'rank', 'th', 'inspir', 'tri', 'golf', 'find', 'get', 'golf', 'special', 'guid', 'champion', 'wood', 'back', 'match', 'play', 'first', 'time', 'sinc', 'three', 'birdi', 'row', 'th

['record_0', '  Bjerregaard beat Henrik Stenson earlier on Saturday Debutant Lucas Bjerregaard carded five birdies and an eagle to end the hopes of Tiger Woods at the WGC Match Play. Woods had beaten Rory McIlroy earlier in the day but missed a four-foot par putt as Bjerregaard won one-up. In Sunday\'s semi-finals world number 52 Bjerregaard faces Matt Kuchar, who beat Sergio Garcia two up after controversy over a putt that was not conceded. Open champion Francesco Molinari surged to a 6&5 win over Kevin Na, who earlier ousted world number two Justin Rose. The Italian will play Kevin Kisner, who won three holes in a row to beat Louis Oosthuizen 2&1. Denmark\'s Bjerregaard, winner of the Dunhill Links last year for his second European Tour title, birdied the first hole to take the lead against Woods, ranked 14th. Inspired to try golf?  Find out how to get into golf with our special guide . Three-time champion Woods, back at the Match Play for the first time since 2013, had three birdies